In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [2]:
from src.federated_pipeline import Federated

In [3]:
federated = Federated(
    "CONFIG_MODEL.json",
    "CONFIG_FEDERATED.json"
)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [ ]:
federated.train(5)

 30%|███       | 3/10 [00:00<00:00, 28.82it/s]

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


 20%|██        | 2/10 [00:02<00:08,  1.01s/it]

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


 40%|████      | 4/10 [00:04<00:07,  1.31s/it]

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0


 80%|████████  | 8/10 [00:11<00:03,  1.59s/it]

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


 73%|███████▎  | 944/1295 [00:03<00:01, 243.98it/s]

In [ ]:
plt.plot(federated.general_model_val_losses.values())

In [ ]:
federated.generate_general('trump')

In [ ]:
federated.generate_node('trump', 1)

In [ ]:
federated.generate_node('trump', 2)

In [ ]:
federated.generate_node('trump', 5)

In [ ]:
federated.generate_node('trump', 7)

In [ ]:
assert False

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

In [ ]:
d = 2
N = 100

mu_0, sigma_0 = [-1]*N, 0.5
mu_1, sigma_1 = [ 1]*N, 0.5

In [ ]:
data_0 = sigma_0 * np.random.randn(d, N) + mu_0
data_1 = sigma_1 * np.random.randn(d, N) + mu_1

labels_0 = [0] * N
labels_1 = [1] * N

In [ ]:
data = np.concatenate((data_0.T, data_1.T))
labels = np.concatenate((labels_0, labels_1))

In [ ]:
class SimpleDataset(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = np.column_stack((data.shape[0] * [1], data))
        self.labels = labels
        
    def __getitem__(self, idx):
        return torch.FloatTensor(self.data[idx]), torch.FloatTensor([self.labels[idx]])
    
    def __len__(self):
        return len(self.labels)
    
dataset = SimpleDataset(data, labels)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size = 4,
    shuffle = True,
    drop_last = True
)

In [ ]:
plt.scatter(data_0[0,:], data_0[1,:])
plt.scatter(data_1[0,:], data_1[1,:])
plt.show()

In [ ]:
class TestModel(torch.nn.Module):
    def __init__(self, length):
        super().__init__()
        self.length = length
        self.vector = torch.nn.Parameter(
            torch.tensor([-0.5] * length),
            requires_grad = True
        )
        
    def forward(self, x):
        x = torch.matmul(x, self.vector.reshape(self.length, -1))
        return torch.sigmoid(x)
        
    def train(self, dataloader, epochs):
        criterion = torch.nn.BCELoss()
        optimizer = torch.optim.SGD(self.parameters(), lr=0.01, momentum=0.9)
        
        for i in range(epochs):
            for x,y in dataloader:
                x = self(x)
                loss = criterion(x, y)
                loss.backward()
                optimizer.step()
            
    def evaluate(self, dataloader):
        acc = []
        with torch.no_grad():
            for x,y in dataloader:
                x = (self.forward(x) > 1/2)
                acc.append(torch.abs(y - x * 1).sum().item() / dataloader.batch_size)
        return np.mean(acc)

In [ ]:
test_model = TestModel(d + 1)
test_model.evaluate(dataloader)

In [ ]:
test_model.train(dataloader, 5)

In [ ]:
test_model.evaluate(dataloader)

In [ ]:
print(test_model.vector)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(test_model.parameters(), lr=0.001, momentum=0.9)
for x,y in dataloader:
    x = test_model.forward(x)
    print(test_model.vector)
    loss = criterion(x,y)
    print(test_model.vector)
    loss.backward()
    print(test_model.vector)
    optimizer.step()
    print(test_model.vector)
    break

In [ ]:
import pickle

In [ ]:
from src.federated_pipeline import NullByzantineNode

In [ ]:
with open('vocab.pickle', 'rb') as f:
    vocabulary = pickle.load(f)

In [ ]:
parameters = {
    'id_' : 1,
    'lambda_' : 1e-3,
    'p' : 1,
    'vocabulary' : vocabulary,
    'min_seq_length' : 2,
    'max_seq_length' : 20,
    'device' : 'cuda:0'
}

nbn = NullByzantineNode(
    **parameters
)

In [ ]:
nbn.data[0]